In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "projects/smtlink/top" :dir :system)
(include-book "centaur/fty/top" :dir :system)


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "projects/smtlink/top" ...):
The ttag note just printed to the terminal indicates a modification
to ACL2.  To avoid this warning, supply an explicit :TTAGS argument
when including the book "/home/acl2/books/projects/smtlink/top".


TTAG NOTE (for included book): Adding ttag :QUICKLISP.SHELLPOOL from book /home/acl2/books/quicklisp/shellpool.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "projects/smtlink/top" ...):
The ttag note just printed to the terminal indicates a modification
to ACL2.  To avoid this warning, supply an explicit :TTAGS argument
when including the book "/home/acl2/books/projects/smtlink/top".


TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/run.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "projects/smtlink/top" ...):
The ttag note just printed to the terminal indicates a modification
t

In [3]:
; Enable tshell for running Z3
(value-triple (tshell-ensure))

 NIL


In [4]:
;; Agent state as FTY product type
;; Using integerp for SMTLink compatibility (maps to Z3 Int)
(fty::defprod agent-state
  ((iteration      integerp :default 0)
   (max-iterations integerp :default 10)
   (token-budget   integerp :default 10000)
   (cost-budget    integerp :default 1000)   ; millicents
   (time-budget    integerp :default 60000)  ; ms
   (file-access    integerp :default 0)      ; 0=none, 1=read, 2=read-write
   (exec-allowed   booleanp :default nil)
   (satisfaction   integerp :default 0)      ; 0-100 (percentage)
   (done           booleanp :default nil))
  :parents (experiment-02))

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [ ]:
;; Tool spec as FTY product type
(fty::defprod tool-spec
  ((required-access integerp :default 0)
   (requires-exec   booleanp :default nil)
   (base-cost       integerp :default 0)
   (time-estimate   integerp :default 0)
   (token-estimate  integerp :default 0))
  :parents (experiment-02))

In [ ]:
(defconst *access-none* 0)
(defconst *access-read* 1)
(defconst *access-read-write* 2)

In [ ]:
(defconst *min-llm-tokens* 100)
(defconst *min-iteration-cost* 10)
(defconst *min-iteration-time* 1000)
(defconst *satisfaction-threshold* 90)  ; percentage

In [ ]:
;; Check if granted access is sufficient
;; Note: We access FTY fields directly - they're already integers
(define access-sufficient-p ((required integerp) (granted integerp))
  :returns (ok booleanp)
  (<= (ifix required) (ifix granted)))

In [ ]:
;; Full permission check: file access + execute
(define tool-permitted-p ((tool tool-spec-p) (st agent-state-p))
  :returns (ok booleanp)
  (b* ((required-access (tool-spec->required-access tool))
       (requires-exec (tool-spec->requires-exec tool))
       (granted-access (agent-state->file-access st))
       (exec-allowed (agent-state->exec-allowed st)))
    (and (access-sufficient-p required-access granted-access)
         (implies requires-exec exec-allowed))))

In [ ]:
;; Check if budget is sufficient for a tool call
(define tool-budget-sufficient-p ((tool tool-spec-p) (st agent-state-p))
  :returns (ok booleanp)
  (b* ((tool-cost (tool-spec->base-cost tool))
       (tool-time (tool-spec->time-estimate tool))
       (tool-tokens (tool-spec->token-estimate tool))
       (budget-cost (agent-state->cost-budget st))
       (budget-time (agent-state->time-budget st))
       (budget-tokens (agent-state->token-budget st)))
    (and (<= tool-cost budget-cost)
         (<= tool-time budget-time)
         (<= tool-tokens budget-tokens))))

In [ ]:
;; Can we invoke this tool?
(define can-invoke-tool-p ((tool tool-spec-p) (st agent-state-p))
  :returns (ok booleanp)
  (and (tool-permitted-p tool st)
       (tool-budget-sufficient-p tool st)))

In [ ]:
;; Must respond: out of budget or done
(define must-respond-p ((st agent-state-p))
  :returns (must booleanp)
  (b* ((done (agent-state->done st))
       (iter (agent-state->iteration st))
       (max-iter (agent-state->max-iterations st))
       (tokens (agent-state->token-budget st))
       (cost (agent-state->cost-budget st))
       (time-ms (agent-state->time-budget st)))
    (or done
        (>= iter max-iter)
        (< tokens *min-llm-tokens*)
        (< cost *min-iteration-cost*)
        (< time-ms *min-iteration-time*))))

In [ ]:
;; Should continue: has budget and below satisfaction threshold
(define should-continue-p ((st agent-state-p))
  :returns (cont booleanp)
  (b* ((sat (agent-state->satisfaction st)))
    (and (not (must-respond-p st))
         (< sat *satisfaction-threshold*))))

In [ ]:
;; Deduct tool costs from state
;; Note: Use (max 0 ...) to ensure non-negative, avoiding ifix
(define deduct-tool-cost ((tool tool-spec-p) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((tool-cost (tool-spec->base-cost tool))
       (tool-time (tool-spec->time-estimate tool))
       (tool-tokens (tool-spec->token-estimate tool))
       (budget-cost (agent-state->cost-budget st))
       (budget-time (agent-state->time-budget st))
       (budget-tokens (agent-state->token-budget st))
       ;; Use max 0 to ensure non-negative (max is supported by SMTLink)
       (new-cost (max 0 (- budget-cost tool-cost)))
       (new-time (max 0 (- budget-time tool-time)))
       (new-tokens (max 0 (- budget-tokens tool-tokens))))
    (change-agent-state st
                        :cost-budget new-cost
                        :time-budget new-time
                        :token-budget new-tokens)))

In [ ]:
;; Increment iteration
(define increment-iteration ((st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st
                      :iteration (1+ (agent-state->iteration st))))

In [ ]:
;; Update satisfaction score
(define update-satisfaction ((new-sat integerp) (st agent-state-p))
  :returns (new-st agent-state-p)
  ;; Clamp to 0-100 using max/min (SMTLink-compatible)
  (b* ((clamped (max 0 (min 100 new-sat))))
    (change-agent-state st :satisfaction clamped)))

In [ ]:
;; Mark done
(define mark-done ((st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st :done t))

In [ ]:
;; Theorem 1: Permission safety (can-invoke implies permitted)
;; This is proven by Z3 via SMTLink
(defthm permission-safety-smt
  (implies (and (tool-spec-p tool)
                (agent-state-p st)
                (can-invoke-tool-p tool st))
           (tool-permitted-p tool st))
  :hints (("Goal"
           :smtlink
           (:fty (tool-spec agent-state)))))

In [ ]:
;; Theorem 2: Budget bounds - deduction keeps budgets non-negative
;; Using >= 0 constraints as these are within the FTY-understood types
(defthm budget-bounds-smt
  (implies (and (tool-spec-p tool)
                (agent-state-p st)
                (>= (agent-state->cost-budget st) 0)
                (>= (agent-state->time-budget st) 0)
                (>= (agent-state->token-budget st) 0))
           (and (>= (agent-state->cost-budget (deduct-tool-cost tool st)) 0)
                (>= (agent-state->time-budget (deduct-tool-cost tool st)) 0)
                (>= (agent-state->token-budget (deduct-tool-cost tool st)) 0)))
  :hints (("Goal"
           :smtlink
           (:fty (tool-spec agent-state)))))

In [ ]:
;; Theorem 3: Iteration bound guarantees termination
(defthm termination-bound-smt
  (implies (and (agent-state-p st)
                (>= (agent-state->iteration st) (agent-state->max-iterations st)))
           (must-respond-p st))
  :hints (("Goal"
           :smtlink
           (:fty (agent-state)))))

In [ ]:
;; Theorem 4: Continue/respond are mutually exclusive and exhaustive
;; (when satisfaction is properly bounded)
(defthm continue-respond-partition-smt
  (implies (and (agent-state-p st)
                (>= (agent-state->satisfaction st) 0)
                (<= (agent-state->satisfaction st) 100))
           (or (must-respond-p st)
               (should-continue-p st)
               (>= (agent-state->satisfaction st) *satisfaction-threshold*)))
  :hints (("Goal"
           :smtlink
           (:fty (agent-state)))))

In [ ]:
(defthm iteration-increases
  (implies (agent-state-p st)
           (< (agent-state->iteration st)
              (agent-state->iteration (increment-iteration st))))
  :hints (("Goal" :in-theory (enable increment-iteration))))

In [ ]:
(defthm deduct-preserves-agent-state
  (implies (and (tool-spec-p tool)
                (agent-state-p st))
           (agent-state-p (deduct-tool-cost tool st)))
  :hints (("Goal" :in-theory (enable deduct-tool-cost))))

In [ ]:
(defthm update-satisfaction-preserves-agent-state
  (implies (and (integerp new-sat)
                (agent-state-p st))
           (agent-state-p (update-satisfaction new-sat st)))
  :hints (("Goal" :in-theory (enable update-satisfaction))))

In [ ]:
(defthm mark-done-preserves-agent-state
  (implies (agent-state-p st)
           (agent-state-p (mark-done st)))
  :hints (("Goal" :in-theory (enable mark-done))))

In [ ]:
(defthm increment-preserves-agent-state
  (implies (agent-state-p st)
           (agent-state-p (increment-iteration st)))
  :hints (("Goal" :in-theory (enable increment-iteration))))